In [ ]:
import pandas as pd
import os

# Paths to the CSV files and image directories
csv_paths = {
    'Train': 'G:\\new dataset\\Disaster_train.csv',
    'Test': 'G:\\new dataset\\Disaster_test.csv',
    'Validation': 'G:\\new dataset\\Disaster_val.csv'
}

image_dirs = {
    'Train': 'G:\\new dataset\\Train',
    'Test': 'G:\\new dataset\\Test',
    'Validation': 'G:\\new dataset\\Validation'
}

output_dir = 'G:\\new dataset\\Output'  # Output directory to save the CSV files

# Function to check for matching Meme_ID and image files, and add image paths
def check_matches(csv_path, image_dir):
    df = pd.read_csv(csv_path)
    image_files = os.listdir(image_dir)
    image_names = {os.path.splitext(image_file)[0]: os.path.join(image_dir, image_file) for image_file in image_files}
    
    # Add Image_Path column to the dataframe
    df['Image_Path'] = df['image_id'].apply(lambda x: image_names.get(x, None))
    
    # Filter rows where Image_Path is not None (i.e., matched Meme_IDs)
    matched_df = df[df['Image_Path'].notna()]
    
    return matched_df

# Check matches for each set (Train, Test, Validation)
for key in csv_paths:
    matched_df = check_matches(csv_paths[key], image_dirs[key])
    
    matches_output_path = os.path.join(output_dir, f'{key}_matches.csv')
    
    matched_df.to_csv(matches_output_path, index=False)
    
    print(f"{key} set:")
    print(f"Matched Meme_IDs with image paths saved to {matches_output_path}\n")


In [ ]:
train_df = pd.read_csv('G:\\new dataset\\Output\\Train_matches.csv')
train_df

In [ ]:
train_df["category"].value_counts()

In [ ]:
test_df = pd.read_csv('G:\\new dataset\\Output\\Test_matches.csv')
test_df

In [ ]:
validation_df = pd.read_csv('G:\\new dataset\\Output\\Validation_matches.csv')
validation_df

In [ ]:
# Display the sizes of each split
print(f"Train dataset size: {len(train_df)}")
print(f"Test dataset size: {len(test_df)}")
print(f"Validation dataset size: {len(validation_df)}")

In [ ]:
train_df["category"].value_counts()

In [ ]:
test_df["category"].value_counts()

In [ ]:
validation_df["category"].value_counts()

In [ ]:
import pandas as pd
import re
import string

# Function to remove punctuation (preserve Bangla characters)
def remove_punctuation(text):
    if pd.isna(text):
        return text
    return text.translate(str.maketrans('', '', string.punctuation))

# Function to remove extra whitespace
def remove_whitespace(text):
    if pd.isna(text):
        return text
    return " ".join(text.split())

# Function to remove emojis
def remove_emojis(text):
    if pd.isna(text):
        return text
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Function to remove URLs
def remove_urls(text):
    if pd.isna(text):
        return text
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

# Function to remove HTML tags
def remove_html(text):
    if pd.isna(text):
        return text
    html_pattern = re.compile(r'<.*?>')
    return html_pattern.sub(r'', text)

# Function to remove special characters (preserve Bangla characters)
def remove_special_characters(text):
    if pd.isna(text):
        return text
    return re.sub(r'[^A-Za-z0-9\s\u0980-\u09FF]', '', text)

# Combine all cleaning functions
def clean_text(text):
    text = remove_urls(text)
    text = remove_html(text)
    text = remove_emojis(text)
    text = remove_punctuation(text)
    text = remove_special_characters(text)
    text = remove_whitespace(text)
    return text

# Mapping categories to integers
category_mapping = {
    'Landslides': 0,
    'Wildfire': 1,
    'Drought': 2,
    'Flood': 3,
    'Earthquake': 4,
    'Tropical Storm': 5,
    'Non Disaster': 6,
    'Human Damage': 7,
}

# Load and clean the dataframes
csv_paths = {
    'Train': 'G:\\new dataset\\Output\\Train_matches.csv',
    'Test': 'G:\\new dataset\\Output\\Test_matches.csv',
    'Validation': 'G:\\new dataset\\Output\\Validation_matches.csv'
}

cleaned_output_paths = {
    'Train': 'G:\\new dataset\\Output\\Train_matches_cleaned.csv',
    'Test': 'G:\\new dataset\\Output\\Test_matches_cleaned.csv',
    'Validation': 'G:\\new dataset\\Output\\Validation_matches_cleaned.csv'
}

text_columns = ['context', 'category']

for key in csv_paths:
    # Load the dataframe
    df = pd.read_csv(csv_paths[key])
    
    # Apply cleaning to all relevant text columns
    for column in text_columns:
        df[column] = df[column].astype(str).apply(clean_text)
    
    # Map 'category' column to integers
    df['category'] = df['category'].map(category_mapping)
    
    # Add 'label' column (same as 'category' for now)
    df['label'] = df['category']
    
    # Display the cleaned dataframe
    print(f"Cleaned {key} dataframe:")
    print(df.head())
    
    # Save the cleaned dataframe to a new CSV file
    df.to_csv(cleaned_output_paths[key], index=False)
    print(f"Cleaned dataframe saved to {cleaned_output_paths[key]}\n")


In [ ]:
train_df = pd.read_csv('G:\\new dataset\\Output\\Train_matches_cleaned.csv')
train_df

In [ ]:
test_df = pd.read_csv('G:\\new dataset\\Output\\Test_matches_cleaned.csv')
test_df

In [ ]:
validation_df = pd.read_csv('G:\\new dataset\\Output\\Validation_matches_cleaned.csv')
validation_df

In [ ]:
validation_df["label"].value_counts()

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from transformers import BertTokenizer, BertModel, AdamW
import torchvision.models as models
from tqdm import tqdm

In [ ]:
# !pip install --upgrade transformers

In [ ]:
from transformers import AutoImageProcessor, EfficientFormerModel
import torch
image_processor = AutoImageProcessor.from_pretrained("snap-research/efficientformer-l1-300")
model = EfficientFormerModel.from_pretrained("snap-research/efficientformer-l1-300")

In [ ]:
from transformers import AutoTokenizer, XLMRobertaModel, AdamW
# Initialize BERT tokenizer and model
bert_tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
bert_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")

In [ ]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
import os

# Enable device-side assertions
os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [ ]:
model.to(device)

In [ ]:
bert_model.to(device)

In [ ]:
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset


max_seq_length = 512  # Set your desired maximum sequence length for BERT

# Define the pre-processing transformations for images
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class MyMultimodalDataset(Dataset):
    def __init__(self, data, transform=None, tokenizer=None, max_seq_length=512):
        self.data = data
        self.transform = transform
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data.iloc[idx]['Image_Path']
        #print(image_path)
        try:
            image = Image.open(image_path).convert('RGB')
            if self.transform is not None:
                image = self.transform(image)
        except Exception as e:
            print(f"Error loading image at index {idx}: {e}")
            return None, None, None, None

        if image is None:
            print(f"Error loading image at idx for None {idx}: {image_path}")
            
            return None, None, None, None

        context = self.data.iloc[idx]['context']

        inputs = self.tokenizer(context, padding='max_length', truncation=True, max_length=self.max_seq_length, return_tensors='pt')
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']

        label = self.data.iloc[idx]['label']

        return image, input_ids, attention_mask, label



In [ ]:
# Create custom datasets with MyMultimodalDataset
train_dataset = MyMultimodalDataset(train_df, transform=transform, tokenizer=bert_tokenizer, max_seq_length=max_seq_length)
test_dataset = MyMultimodalDataset(test_df, transform=transform, tokenizer=bert_tokenizer, max_seq_length=max_seq_length)
val_dataset = MyMultimodalDataset(validation_df, transform=transform, tokenizer=bert_tokenizer, max_seq_length=max_seq_length)

# Define data loaders
batch_size = 6  # Set your desired batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# import torch
# import torchvision.models as models
# from transformers import AutoImageProcessor, ViTModel, AutoTokenizer, DistilBertModel, AdamW
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Assuming you have defined your train_loader, val_loader, optimizer, criterion, model, bert_model, etc.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Feature dimensions from ViTModel and DistilBertModel
img_feat_size = 448  # Updated based on the debug print output
text_feat_size = 768
combined_input_dim = img_feat_size + text_feat_size

# Define the regressor model outside the training loop
regressor = torch.nn.Sequential(
    torch.nn.Linear(combined_input_dim, 512),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(512, 8)  # Change output size to match your number of labels
).to(device)

# Combine parameters from both models and the regressor for the optimizer
optimizer = torch.optim.AdamW(list(model.parameters()) + list(bert_model.parameters()) + list(regressor.parameters()), lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 35
train_losses = []
val_losses = []


start_time = time.time()
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
for epoch in range(num_epochs):
    running_train_loss = 0.0
    
    model.train()
    bert_model.train()
    regressor.train()
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    for images, texts, attention_masks, labels in tqdm(train_loader, desc='Training', leave=False):
        images = images.to(device)
        labels = labels.to(device).long()

        input_ids = texts.squeeze(1).to(device)
        attention_mask = attention_masks.squeeze(1).to(device)

        optimizer.zero_grad()

        with torch.no_grad():
            outputs_image = model(pixel_values=images)
        img_hidden_states = outputs_image.last_hidden_state
        img_feats = img_hidden_states[:, 0, :]

        outputs_text = bert_model(input_ids=input_ids, attention_mask=attention_mask)
        text_hidden_states = outputs_text.last_hidden_state
        text_feats = text_hidden_states[:, 0, :]

        combined_feats = torch.cat((img_feats, text_feats), dim=1)

        predictions = regressor(combined_feats).float()
        loss = criterion(predictions, labels)

        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    epoch_train_loss = running_train_loss / len(train_loader)
    train_losses.append(epoch_train_loss)

    model.eval()
    bert_model.eval()
    regressor.eval()

    running_val_loss = 0.0

    with torch.no_grad():
        for val_images, val_texts, val_attention_masks, val_labels in tqdm(val_loader, desc='val', leave=False):
            val_images = val_images.to(device)
            val_labels = val_labels.to(device).long()

            val_input_ids = val_texts.squeeze(1).to(device)
            val_attention_mask = val_attention_masks.squeeze(1).to(device)

            outputs_image = model(pixel_values=val_images)
            val_img_hidden_states = outputs_image.last_hidden_state
            val_img_feats = val_img_hidden_states[:, 0, :]

            outputs_text = bert_model(input_ids=val_input_ids, attention_mask=val_attention_mask)
            val_text_hidden_states = outputs_text.last_hidden_state
            val_text_feats = val_text_hidden_states[:, 0, :]

            val_combined_feats = torch.cat((val_img_feats, val_text_feats), dim=1)

            val_predictions = regressor(val_combined_feats).float()
            val_loss = criterion(val_predictions, val_labels)
            # Debugging prints
#             print("Shapes:")
#             print(f"val_images: {val_images.shape}")
#             print(f"val_labels: {val_labels.shape}")
#             print(f"val_input_ids: {val_input_ids.shape}")
#             print(f"val_attention_mask: {val_attention_mask.shape}")
#             print(f"val_img_feats: {val_img_feats.shape}")
#             print(f"val_text_feats: {val_text_feats.shape}")
#             print(f"val_combined_feats: {val_combined_feats.shape}")
#             print(f"val_predictions: {val_predictions.shape}")
#             print(f"val_loss: {val_loss.item()}")

            running_val_loss += val_loss.item()

    epoch_val_loss = running_val_loss / len(val_loader)
    val_losses.append(epoch_val_loss)

    print(f"Epoch [{epoch + 1}/{num_epochs}] - "
          f"Train Loss: {epoch_train_loss:.4f}, "
          f"Val Loss: {epoch_val_loss:.4f}")

end_time = time.time()
execution_time = end_time - start_time
print(f"Total execution time: {execution_time:.2f} seconds")

In [ ]:
# Prepare lists to store predicted and true labels
predicted_labels = []
true_labels = []

model.eval()
bert_model.eval()
regressor.eval()

running_test_loss = 0.0

with torch.no_grad():
    for test_images, test_texts, test_attention_masks, test_labels in tqdm(test_loader, desc='Testing', leave=False):
        test_images = test_images.to(device)
        test_labels = test_labels.to(device).long()

        test_input_ids = test_texts.squeeze(1).to(device)
        test_attention_mask = test_attention_masks.squeeze(1).to(device)

        outputs_image = model(pixel_values=test_images)
        test_img_hidden_states = outputs_image.last_hidden_state
        test_img_feats = test_img_hidden_states[:, 0, :]

        outputs_text = bert_model(input_ids=test_input_ids, attention_mask=test_attention_mask)
        test_text_hidden_states = outputs_text.last_hidden_state
        test_text_feats = test_text_hidden_states[:, 0, :]

        test_combined_feats = torch.cat((test_img_feats, test_text_feats), dim=1)

        test_predictions = regressor(test_combined_feats).float()
        test_loss = criterion(test_predictions, test_labels)

        running_test_loss += test_loss.item()

        # Store predicted and true labels
        predicted_labels.extend(test_predictions.argmax(dim=1).cpu().numpy())
        true_labels.extend(test_labels.cpu().numpy())

epoch_test_loss = running_test_loss / len(test_loader)
print(f"Test Loss: {epoch_test_loss:.4f}")

# Convert lists to numpy arrays for further evaluation
predicted_labels = np.array(predicted_labels)
true_labels = np.array(true_labels)

# You can now use predicted_labels and true_labels for further evaluation
# For example, calculating accuracy, precision, recall, etc.
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")


In [ ]:
predicted_labels 

In [ ]:
true_labels 

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, mean_squared_error, classification_report


# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate precision, recall, F1-score overall (macro average)
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='macro')

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Calculate Mean Squared Error
mse = mean_squared_error(true_labels, predicted_labels)

# Calculate Sensitivity (Recall) for each class
sensitivity_per_class = recall

# Calculate Specificity for each class
specificity_per_class = []
for i in range(len(conf_matrix)):
    tn = np.sum(conf_matrix) - (np.sum(conf_matrix[i, :]) + np.sum(conf_matrix[:, i]) - conf_matrix[i, i])
    fp = np.sum(conf_matrix[:, i]) - conf_matrix[i, i]
    specificity_per_class.append(tn / (tn + fp))

# Print overall calculated metrics
print(f"Accuracy: {accuracy}")
print(f"Precision (macro): {precision}")
print(f"Recall (macro): {recall}")
print(f"F1-Score (macro): {f1_score}")
print(f"Mean Squared Error: {mse}")

# Print Sensitivity and Specificity for each class
print(f"Sensitivity (Recall) for each class: {sensitivity_per_class}")
print(f"Specificity for each class: {specificity_per_class}")


In [ ]:
# Mapping categories to integers
class_names = [
    'Landslides',
    'Wildfire',
    'Drought',
    'Flood',
    'Earthquake',
    'Tropical Storm',
    'Non Disaster',
    'Human Damage']


In [ ]:
conf_matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Plot confusion matrix
plt.figure(figsize=(8, 6))
# Class names according to the label encoding mapping

sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='d',xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()